In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.RNN_Transaction_Dataset import RNN_Transaction_Dataset

from Model.LSTM import LSTM

In [2]:
input_size = 5
hidden_size = 256
output_size = 1

lr = 1e-3
num_epochs = 1

In [3]:
transaction_df = pd.read_excel('../데이터/Transaction/transaction_final.xlsx', index_col=0)
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')
economy_df = economy_df['국고채금리']

In [4]:
trainsaction_train_size = int(len(transaction_df)*0.7)
trainsaction_val_size = int(len(transaction_df)*0.3)

transaction_train_dataset = RNN_Transaction_Dataset(transaction_df[:trainsaction_train_size])
transaction_train_loader = DataLoader(transaction_train_dataset, batch_size=2)
transaction_val_dataset = RNN_Transaction_Dataset(transaction_df[trainsaction_train_size:])
transaction_val_loader = DataLoader(transaction_val_dataset, batch_size=2)

economy_train_size = int(len(economy_df)*0.7)
economy_val_size = int(len(economy_df)*0.3)

economy_train_dataset = Economy_Dataset(economy_df[:economy_train_size])
economy_train_loader = DataLoader(economy_train_dataset, batch_size=2)
economy_val_dataset = Economy_Dataset(economy_df[economy_train_size:])
economy_val_loader = DataLoader(economy_val_dataset, batch_size=2)

c:\Users\USER\Desktop\sci\SCI\코드\Dataset\RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])
c:\Users\USER\Desktop\sci\SCI\코드\Dataset\RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])
c:\Users\USER\Desktop\sci\SCI\코드\Dataset\Economy_Dataset.py:11: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe (Triggered 

In [5]:
model = LSTM(input_size, hidden_size, output_size)

In [6]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 경제 모델

In [16]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(economy_train_loader):
        economy_x_train, economy_y_train = samples

        prediction, hidden = model(economy_x_train)
        cost = criterion(prediction, economy_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(economy_val_loader):
            economy_x_val, economy_y_val = samples

            prediction, hidden = model(economy_x_val)
            loss = criterion(prediction, economy_y_val)
            val_loss += loss.item()

    val_loss /= len(economy_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_economy_model.pth')

Epoch 0/1, Training Loss: 0.002730900188907981, Validation Loss: 0.3143093644015106
Epoch 1/1, Training Loss: 0.0027428490575402975, Validation Loss: 0.31638226120186774


### 부동산 모델

In [7]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(transaction_train_loader):
        dong_x_train, dong_y_train = samples

        prediction, hidden = model(dong_x_train)
        cost = criterion(prediction, dong_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(transaction_val_loader):
            x_val, y_val = samples

            prediction, hidden = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(transaction_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_transaction_model.pth')

Epoch 0/1, Training Loss: 28219.5234375, Validation Loss: 5213844.961625429
Epoch 1/1, Training Loss: 27884.662109375, Validation Loss: 5179137.586078938
